In [1]:
import numpy as np
import open3d as o3d
import pickle
from os import path
import matplotlib.pyplot as plt
import time


In [2]:
import sys 
from os import path
sys.path.append(path.relpath('./'))
sys.path.append(path.relpath('./CenterPoint'))
from CenterPoint.tools.visual import center_to_corner_box3d, label2color, corners_to_lines

Import spconv fail, no support for sparse convolution!
iou3d cuda not built. You don't need this if you use circle_nms. Otherwise, refer to the advanced installation part to build this cuda extension


In [3]:
with open("waymo/detection_pred.pkl", 'rb') as f:
    detection_preds_dict = pickle.load(f)

In [4]:
#frame_name = "39847154216997509_6440_000_6460_000-1568954807924818"
#frame_name = "6503078254504013503_3440_000_3460_000-1557855930472609"
#frame_name = "7240042450405902042_580_000_600_000-1559312894537967"

In [5]:
#frame_name = "39847154216997509_6440_000_6460_000-1568954821724274"
frame_name = "10488772413132920574_680_000_700_000-1557276680370783"

In [6]:
pred_detections = detection_preds_dict[frame_name]

In [7]:
pred_detections["label_preds"]

array([1., 1., 2., 1., 1., 2., 2., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1.,
       2., 2., 1., 2., 2., 1., 2., 2., 2., 4., 1., 2., 1., 2., 2., 2., 2.,
       2., 2., 1., 2., 1., 1., 2., 2., 2., 1., 2., 2., 1., 2., 2., 2., 1.,
       2., 1., 1., 2., 1., 2., 2., 2., 2., 1., 2., 2., 1., 2., 1., 2., 2.,
       2., 2., 1., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2.,
       2., 1., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2.,
       2., 2., 2., 1., 2., 1., 2., 2., 2., 2., 2., 1., 2., 2., 1., 1., 2.,
       1., 1., 2., 2., 2., 1., 1., 1., 2., 2., 1., 2., 2., 2., 2., 1., 2.,
       1., 2., 1., 2., 2., 2., 1., 1., 1., 1., 1., 2., 1., 1., 2., 2., 2.,
       1., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 1., 2., 2., 1., 2.,
       2., 1., 2., 1., 2., 2., 2., 1., 1., 2., 1., 2., 2., 1., 2., 2., 2.,
       2., 1., 2., 2., 2., 2., 1., 1., 2., 2., 1., 2., 2., 2., 1., 4., 2.,
       1., 4., 2., 2., 1., 2., 1., 2., 1., 1., 1., 2., 2., 2., 2., 2., 2.,
       2., 1., 4., 2., 2.

In [8]:
pc_name = path.join("../results/lidar", frame_name +".pkl")
with open(pc_name, 'rb') as f:
    lidar_dict = pickle.load(f)

In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(lidar_dict["lidars"]["points_xyz"])
pcd.paint_uniform_color([0.8,0.8,0.8])

PointCloud with 176258 points.

In [10]:
cluster_name = path.join("../results/cluster", frame_name +".pkl")
with open(cluster_name, 'rb') as f:
    cluster_dict = pickle.load(f) #x,y,z,cluster,label
labeled_clusters = cluster_dict["cluster_pcs"]

In [11]:
print(cluster_dict["cluster_bboxes"])

[[24.69628906  5.80493164  2.10778809  0.2578125   0.32324219  1.86523438
  -1.        ]
 [ 2.          2.          2.          2.          2.          2.
   2.        ]
 [26.51269531  1.54016113  2.17990112  2.54296875  1.67211914  1.88336182
  -1.        ]
 [ 1.          1.          1.          1.          1.          1.
   1.        ]
 [ 1.          1.          1.          1.          1.          1.
   1.        ]
 [ 1.          1.          1.          1.          1.          1.
   1.        ]
 [ 4.          4.          4.          4.          4.          4.
   4.        ]
 [ 1.          1.          1.          1.          1.          1.
   1.        ]]


In [12]:
clusters = np.delete(labeled_clusters, -1, axis=1)

In [13]:
with open(path.join("../results/road_mask", frame_name + ".pkl"), "rb") as f:
    road_points = pickle.load(f)
    road = o3d.geometry.PointCloud()
    road.points = o3d.utility.Vector3dVector(road_points)

## Visualize clustering

In [14]:
sorted_clusters = clusters[clusters[:,-1].argsort()]
np.unique(sorted_clusters[:,-1])
cluster_list = np.split(sorted_clusters[:,:-1], np.unique(sorted_clusters[:,-1], return_index=True)[1][1:])

In [15]:
cluster_bboxes = [o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster)) for cluster in cluster_list]

In [16]:
cluster_line_sets = [o3d.geometry.LineSet().create_from_axis_aligned_bounding_box(box) for box in cluster_bboxes]
for lineset in cluster_line_sets:
    lineset.colors = o3d.utility.Vector3dVector([[0, 0,255] for i in range(12)])

In [17]:
clusters_pc = o3d.geometry.PointCloud()
clusters_pc.points = o3d.utility.Vector3dVector(clusters[:,:-1])
cluster_labels = clusters[:,-1]
max_label = cluster_labels.max()
print(f"point cloud has {int(max_label + 1)} clusters")
colors = plt.get_cmap("tab20b")(cluster_labels / (max_label if max_label > 0 else 1))
colors[cluster_labels < 0] = 0
clusters_pc.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 8 clusters


In [18]:
SCORE_THRESHOLD = 0.5

In [19]:
def plot_boxes(predictions, score_thresh):
    visuals =[] 
    num_det = predictions['scores'].shape[0]
    for i in range(num_det):
        score = predictions['scores'][i]
        if score < score_thresh:
            continue 

        box = predictions['box3d_lidar'][i:i+1]#.numpy()
        print(box)
        label = predictions['label_preds'][i]
        corner = center_to_corner_box3d(box[:, :3], box[:, 3:6], box[:, -1])[0].tolist()
        color = label2color(int(label) -1)
        visuals.append(corners_to_lines(corner, color))
    return visuals

In [20]:
visuals = [pcd, clusters_pc]
visuals += cluster_line_sets
pred_box_linesets = plot_boxes(pred_detections, SCORE_THRESHOLD)
visuals += pred_box_linesets
o3d.visualization.draw_geometries(visuals, width=3072, height=1920)

[[-10.911776    -5.2522826    0.691644     4.791518     2.1338737
    1.8710948    0.           0.          -0.07108343]]
[[-17.130545   -4.933069    0.65242     4.554404    2.1105826   1.77434
    0.          0.         -0.1250366]]
[[27.630629   5.0141616  2.228476   1.0646409  1.0059998  1.8364768
   0.         0.        -0.8908042]]
[[30.6671     -9.860727    1.7350397   4.7381725   2.050109    1.5572492
   0.          0.         -0.10080063]]
[[-1.8742723   3.1016316   0.7168346   4.861818    2.147382    1.5226868
   0.          0.         -0.10638916]]
[[ 8.603115    6.716971    1.1336703   1.12751     0.88702303  1.8520865
   0.          0.         -1.7113246 ]]
[[55.630978    1.3010044   3.3949757   0.8490493   0.78018284  1.6657428
   0.          0.         -3.2755942 ]]
[[-41.793888    -2.5826147    0.47546053   4.3582363    1.9391575
    1.5039326    0.           0.          -0.09992719]]
[[-25.731527     9.978166     1.1634897    1.066333     0.88573784
    1.7909464    0. 

In [21]:
def center_to_AABB_box3d(boxes):
    AABBs =[] 
    num_det = len(boxes)
    for i in range(num_det):
        box = boxes[i:i+1]
        corner = np.array(center_to_corner_box3d(box[:, :3], box[:, 3:6], box[:, -1])[0].tolist())
        AABBs.append(o3d.geometry.AxisAlignedBoundingBox(corner.min(axis=0), corner.max(axis=0)))
    return AABBs

In [22]:
pred_detections["box3d_lidar"].shape

(376, 9)

In [23]:
pred_bboxes_points = pred_detections["box3d_lidar"][np.argwhere(pred_detections["scores"] > SCORE_THRESHOLD).flatten()]

In [24]:
AABBs = center_to_AABB_box3d(pred_bboxes_points)

In [25]:
PIBB_THRESHOLD = 0.5

In [26]:
for lineset in pred_box_linesets:
    lineset.colors = o3d.utility.Vector3dVector([[0, 0, 0] for i in range(12)])

In [27]:
start = time.time()
cluster_pc_list = []
for cluster_index, cluster_points in zip(range(len(cluster_list)), cluster_list):    
    cluster_pc = o3d.geometry.PointCloud()
    cluster_pc.points = o3d.utility.Vector3dVector(cluster_points)
    cluster_pc.paint_uniform_color([1,0,0])
    for AABB_index, AABB in zip(range(len(AABBs)), AABBs):
        num_points = len(cluster_points)
        points_in_AABB_index = AABB.get_point_indices_within_bounding_box(o3d.utility.Vector3dVector(cluster_points))
        if((len(points_in_AABB_index) / num_points) >= PIBB_THRESHOLD):
            colors = np.asarray(cluster_pc.colors)
            colors[points_in_AABB_index] = [0, 255, 0]
            cluster_pc.colors = o3d.utility.Vector3dVector(colors)
            cluster_pc_list.append(cluster_pc)
            
            pred_box_linesets[AABB_index].colors = o3d.utility.Vector3dVector([[0, 255, 0] for i in range(12)])
            cluster_line_sets[cluster_index].colors = o3d.utility.Vector3dVector([[0, 255, 0] for i in range(12)])
            break

end = time.time()
print("Took {} seconds".format(end - start))

Took 0.05016899108886719 seconds


In [28]:
visuals = [pcd, road]
visuals += cluster_line_sets
visuals += pred_box_linesets
visuals += cluster_pc_list
o3d.visualization.draw_geometries(visuals, width=3072, height=1920,)